<a href="https://colab.research.google.com/github/mahaksinhal/merit-list-and-scholarship-allocation/blob/main/LEX_and_YACC_Compiler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LEX and YACC Compiler in Colab

Drawbacks:
* Regular interrupts (Ctrl+D, Ctrl+C) for shell won't work in Colab while inputting for program.
<br>Workaround: Store your inputs in a txt file and pass it to the program.

In [8]:
#@title Install *prerqeuisites* (run this cell first to work on LEX/YACC)
!sudo apt install flex bison

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
bison is already the newest version (2:3.8.2+dfsg-1build1).
flex is already the newest version (2.6.4-8build2).
0 upgraded, 0 newly installed, 0 to remove and 41 not upgraded.


In [11]:
!sudo apt-get install flex gcc
!pip install flask pyngrok


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
flex is already the newest version (2.6.4-8build2).
gcc is already the newest version (4:11.2.0-1ubuntu1).
gcc set to manually installed.
0 upgraded, 0 newly installed, 0 to remove and 41 not upgraded.


## Lex only

In [9]:
#@title Writing Lex program
%%writefile dag_edge.l
%{
#include <stdio.h>
#include <stdlib.h>
#include <string.h>

#define MAX 50

// Node structure for DAG
struct Node {
    char op;
    char left[10];
    char right[10];
    int index;
} dag[MAX];

int node_count = 0;

// === Function Declarations ===
int search(char op, char *left, char *right);
void insert(char op, char *left, char *right);
void display_nodes();
void display_edges();
void trim(char *str);

%}

%option noyywrap

%%
[ \t\n]       ;
[a-zA-Z]      { printf("Identifier: %s\n", yytext); }
"+"|"-"|"*"|"/" {
                    char op = yytext[0];
                    printf("Operator: %c\n", op);
                }
.              ;
%%

// === Helper and DAG Functions ===

void trim(char *str) {
    char *p1 = str, *p2 = str;
    while(*p2 != '\0') {
        if(*p2 != ' ')
            *p1++ = *p2;
        p2++;
    }
    *p1 = '\0';
}

int search(char op, char *left, char *right) {
    for (int i = 0; i < node_count; i++) {
        if (dag[i].op == op &&
            strcmp(dag[i].left, left) == 0 &&
            strcmp(dag[i].right, right) == 0)
            return i;
    }
    return -1;
}

void insert(char op, char *left, char *right) {
    int pos = search(op, left, right);
    if (pos == -1) {
        dag[node_count].op = op;
        strcpy(dag[node_count].left, left);
        strcpy(dag[node_count].right, right);
        dag[node_count].index = node_count + 1;
        node_count++;
    }
}

void display_nodes() {
    printf("\n--- DAG Nodes ---\n");
    printf("Index\tOp\tLeft\tRight\n");
    for (int i = 0; i < node_count; i++) {
        printf("%d\t%c\t%s\t%s\n", dag[i].index, dag[i].op, dag[i].left, dag[i].right);
    }
}

void display_edges() {
    printf("\n--- DAG Edges (Parent → Child) ---\n");
    for (int i = 0; i < node_count; i++) {
        printf("Node%d(%c) → %s\n", dag[i].index, dag[i].op, dag[i].left);
        printf("Node%d(%c) → %s\n", dag[i].index, dag[i].op, dag[i].right);
    }
}

int main() {
    char expr[100];
    printf("Enter an expression (e.g., a+b*c+b*c): ");
    scanf("%s", expr);

    trim(expr);

    // Simple parsing: handle left-to-right binary operators
    for (int i = 0; i < strlen(expr); i++) {
        if (expr[i] == '+' || expr[i] == '-' || expr[i] == '*' || expr[i] == '/') {
            char left[10] = { expr[i - 1], '\0' };
            char right[10] = { expr[i + 1], '\0' };
            insert(expr[i], left, right);
        }
    }

    display_nodes();
    display_edges();

    printf("\n--- Lexical Analysis ---\n");
    yylex();

    return 0;
}


Overwriting dag_edge.l


%%html
<!DOCTYPE html>
<html lang="en">
<head>
  <meta charset="UTF-8">
  <title>DAG Visualization for Expression</title>
  <script src="https://cdnjs.cloudflare.com/ajax/libs/p5.js/1.9.0/p5.min.js"></script>
  <style>
    body {
      font-family: 'Segoe UI', sans-serif;
      text-align: center;
      background: #f7f9fb;
    }
    input {
      padding: 10px;
      width: 300px;
      font-size: 16px;
      border-radius: 8px;
      border: 1px solid #ccc;
      margin: 10px;
    }
    button {
      padding: 10px 20px;
      background-color: #0066cc;
      color: white;
      border: none;
      border-radius: 8px;
      cursor: pointer;
    }
    button:hover { background-color: #004c99; }
    canvas { margin-top: 20px; border: 1px solid #ddd; border-radius: 10px; }
  </style>
</head>
<body>
  <h2>Expression → DAG Generator</h2>
  <input type="text" id="exprInput" placeholder="Enter expression (e.g. a+b*c+b*c)" />
  <button onclick="generateDAG()">Generate DAG</button>
  <div id="output"></div>
  <script>
    let dagNodes = [];
    let edges = [];

    function parseExpression(expr) {
      dagNodes = [];
      edges = [];
      expr = expr.replace(/\s+/g, '');

      let nodeCount = 0;
      for (let i = 0; i < expr.length; i++) {
        if ("+-*/".includes(expr[i])) {
          let left = expr[i-1];
          let right = expr[i+1];
          // Check if already exists
          let existing = dagNodes.find(n => n.op===expr[i] && n.left===left && n.right===right);
          if (!existing) {
            nodeCount++;
            dagNodes.push({ index: nodeCount, op: expr[i], left: left, right: right });
            edges.push({ parent: nodeCount, child: left });
            edges.push({ parent: nodeCount, child: right });
          }
        }
      }
    }

    function generateDAG() {
      const expr = document.getElementById('exprInput').value;
      if (!expr) { alert('Please enter an expression!'); return; }
      parseExpression(expr);
      document.getElementById('output').innerHTML = `
        <h3>DAG Nodes</h3>
        <table border="1" cellpadding="8" style="margin:auto;border-collapse:collapse;">
          <tr><th>Index</th><th>Operator</th><th>Left</th><th>Right</th></tr>
          ${dagNodes.map(n => `<tr><td>${n.index}</td><td>${n.op}</td><td>${n.left}</td><td>${n.right}</td></tr>`).join('')}
        </table>
        <div id="dagCanvas"></div>
      `;
      drawDAG();
    }

    function drawDAG() {
      let sketch = (p) => {
        p.setup = () => {
          let canvas = p.createCanvas(600, 400);
          canvas.parent('dagCanvas');
          p.background(255);
          p.textAlign(p.CENTER, p.CENTER);
          p.stroke(0);
          p.textSize(14);

          let x = 100, y = 100;
          dagNodes.forEach((n, i) => {
            let nx = x + i * 150;
            let ny = y;
            p.fill(220);
            p.ellipse(nx, ny, 60, 60);
            p.fill(0);
            p.text(n.op, nx, ny);
            n.x = nx; n.y = ny;
          });

          dagNodes.forEach(n => {
            let leftNode = dagNodes.find(d => d.op === n.left);
            let rightNode = dagNodes.find(d => d.op === n.right);
            if (!leftNode) {
              p.fill('#aaf');
              p.ellipse(n.x - 40, n.y + 120, 40, 40);
              p.fill(0); p.text(n.left, n.x - 40, n.y + 120);
              p.line(n.x, n.y + 30, n.x - 40, n.y + 100);
            }
            if (!rightNode) {
              p.fill('#aaf');
              p.ellipse(n.x + 40, n.y + 120, 40, 40);
              p.fill(0); p.text(n.right, n.x + 40, n.y + 120);
              p.line(n.x, n.y + 30, n.x + 40, n.y + 100);
            }
          });
        };
      };
      new p5(sketch);
    }
  </script>
</body>
</html>


In [15]:
%%html
<!DOCTYPE html>
<html lang="en">
<head>
  <meta charset="UTF-8">
  <title>DAG Frontend (Colab)</title>
  <style>
    body { font-family: 'Segoe UI', sans-serif; text-align: center; background:#eef3f7;}
    input { padding:10px; width:280px; border-radius:8px; border:1px solid #aaa; }
    button { padding:10px 20px; background:#0077cc; color:white; border:none; border-radius:8px; cursor:pointer; }
    pre { background:white; width:80%; margin:auto; padding:20px; border-radius:8px; text-align:left; }
  </style>
</head>
<body>
  <h2>DAG Generator (Linked to Lex + C)</h2>
  <input id="expr" placeholder="Enter expression (e.g. a+b*c+b*c)" />
  <button onclick="runDAG()">Run</button>
  <pre id="output"></pre>

  <script>
    const backend = "from flask import Flask, request, jsonify
from pyngrok import ngrok
import subprocess

app = Flask(__name__)

@app.route("/dag", methods=["POST"])
def dag_route():
    data = request.get_json()
    expr = data.get("expression", "")
    if not expr:
        return jsonify({"error": "Empty expression"}), 400
    try:
        result = subprocess.check_output(["./dag", expr], text=True)
        return jsonify({"output": result})
    except Exception as e:
        return jsonify({"error": str(e)})

# expose the app via ngrok
public_url = ngrok.connect(5000).public_url
print("Frontend can connect here →", public_url)
app.run(port=5000)
";   // Replace with printed ngrok URL

    async function runDAG() {
      const expr = document.getElementById("expr").value;
      document.getElementById("output").innerText = "Processing...";
      const res = await fetch(backend, {
        method: "POST",
        headers: { "Content-Type": "application/json" },
        body: JSON.stringify({ expression: expr })
      });
      const data = await res.json();
      if (data.output)
        document.getElementById("output").innerText = data.output;
      else
        document.getElementById("output").innerText = "Error: " + data.error;
    }
  </script>
</body>
</html>


ERROR:pyngrok.process.ngrok:t=2025-11-08T06:39:10+0000 lvl=eror msg="failed to reconnect session" obj=tunnels.session err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n"


PyngrokNgrokError: The ngrok process errored on start: authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n.

In [ ]:
#@title Shell Execution (you can rewrite the commands as per your need, eg. if you want to include a file as an input)
%%shell

flex dag_edge.l
gcc lex.yy.c -o dag_edges
./dag_edges

Enter an expression (e.g., a+b*c+b*c): a+b*c+b*c

--- DAG Nodes ---
Index	Op	Left	Right
1	+	a	b
2	*	b	c
3	+	c	b

--- DAG Edges (Parent → Child) ---
Node1(+) → a
Node1(+) → b
Node2(*) → b
Node2(*) → c
Node3(+) → c
Node3(+) → b

--- Lexical Analysis ---


CalledProcessError: Command '
flex dag_edge.l
gcc lex.yy.c -o dag_edges
./dag_edges
' died with <Signals.SIGINT: 2>.